In [17]:
# Package Imports

# File management
import os

# Data/numeric manipulation
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

# Modelling & Evaluation
from sklearn.metrics import accuracy_score
import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score


In [3]:
### read in data

beer_dat = pd.read_csv(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\data\raw\beer_reviews.csv")

In [27]:
print("Number of beer styles: {}".format(len(beer_dat.brewery_id.unique())))
beer_dat.brewery_id.unique()

Number of beer styles: 5840


array([10325,  1075,   163, ..., 12566,  7337,  6102], dtype=int64)

In [8]:
# Divide up data
num_cols = ['review_aroma','review_appearance','review_palate','review_taste']

cat_cols = ['brewery_id']
key_feat = ['brewery_id','review_aroma','review_appearance','review_palate','review_taste','beer_style']

df_cleaned = beer_dat[key_feat]

In [9]:
#Instantiate SC
sc = StandardScaler()
#Scale numerics
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])


#scale Cat
X_cat = df_cleaned[cat_cols].astype('category')
df_cleaned.drop(cat_cols, axis=1, inplace=True)
ode = OrdinalEncoder()
X_cat_y = pd.DataFrame(ode.fit_transform(df_cleaned[y_cat_cols]))
X_cat_y.columns = y_cat_cols
X_cat_y = X_cat_y.astype(int)
df_cleaned.drop(y_cat_cols, axis=1, inplace=True)
X_cat_cols = pd.concat([X_cat, X_cat_y ], axis=1)

#recombine as X
X = pd.concat([df_cleaned, X_cat_cols ], axis=1)

X = X.drop(columns=['brewery_id'])

<ipython-input-9-1a50bd7a9b68>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])
c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [10]:
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\data")
from sets import subset_x_y, split_sets_by_time, save_sets, split_sets_random
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks")

In [11]:
### Train test split, change to numpy

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col='beer_style', test_ratio=0.2, to_numpy=True)

In [12]:
# Train RF Model

xgb = XGBClassifier(eval_metric='auc')
xgb.fit(X_train, y_train)
print(xgb)

c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [13]:
### Evaluations
y_test_pred = xgb.predict(X_test)
conf_mat = confusion_matrix(y_test, y_test_pred)

# print evaluation scores
print(f'{np.sum(y_test_pred)} out of {len(y_test_pred)} or {int(round(np.sum(y_test_pred)/len(y_test_pred)*100,0))}% of players are predicted to be 5+ Yrs')
print(f'-----------\nRecall: {round(recall_score(y_test, y_test_pred), 4)}')
print(f'Accuracy: {round(accuracy_score(y_test, y_test_pred), 4)}')
print(f'F1: {round(f1_score(y_test, y_test_pred), 4)}')
print(f'-----------')



ValueError: `data` and `annot` must have same shape.

In [14]:
# Save model
from joblib import dump  

dump(xgb, r'C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\models\XGB_4feat.joblib')

['C:\\Users\\Angus\\Documents\\UTS MDSI\\Advanced DSI\\Projects\\advdsi_at2\\models\\XGB_4feat.joblib']

In [20]:
from sklearn.metrics import classification_report, confusion_matrix


In [22]:
y_test_pred = xgb.predict(X_test)

print(classification_report(y_test, y_test_pred))



c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1521
           1       0.14      0.63      0.23      6085
           2       0.04      0.00      0.00      9288
           3       0.00      0.00      0.00      1887
           4       0.00      0.00      0.00      5390
           5       0.00      0.00      0.00      2394
           6       0.00      0.00      0.00      2594
           7       0.03      0.00      0.00      5066
           8       0.00      0.00      0.00       296
           9       0.11      0.22      0.15     17159
          10       0.00      0.00      0.00      1109
          11       0.08      0.03      0.04     10187
          12       0.08      0.71      0.15     23356
          13       0.00      0.00      0.00       759
          14       0.06      0.09      0.07     12479
          15       0.00      0.00      0.00      1871
          16       0.05      0.00      0.00      4900
          17       0.06    

c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:

cm = confusion_matrix(y_test, y_test_pred)
cm

array([[   0,  129,    3, ...,    0,    0,    1],
       [   0, 3834,   33, ...,    0,    0,    5],
       [   0,  821,   21, ...,    0,    0,    8],
       ...,
       [   0,   20,    1, ...,    0,    0,    0],
       [   0,  225,    5, ...,    0,    0,    3],
       [   0,  600,   17, ...,    1,    0,   11]], dtype=int64)